In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import logging
from tqdm import tqdm
import time
import os
import sys

In [5]:
# Constants
BASE_URL = 'https://jobs.undp.org/cj_view_job.cfm?cur_job_id='
CHECKPOINT_FILE = 'checkpoint.txt'
OUTPUT_FILE = 'undp_jobs.csv'
MIN_JOB_ID = 1150
MAX_RETRIES = 3
RETRY_DELAY = 5

In [8]:
def get_soup(url, retries=MAX_RETRIES):
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=10)
            return BeautifulSoup(response.content, "html.parser")
        except requests.RequestException as e:
            logging.warning(f"Error fetching {url}: {e}. Retrying...")
            time.sleep(RETRY_DELAY)
    logging.error(f"Failed to fetch {url} after {MAX_RETRIES} retries")
    return None

In [12]:
def is_valid_job_posting(url):
    soup = get_soup(url)
    if not soup:
        return False
    # Check if the page contains typical job posting content
    return "cj_view_job.cfm" in url

In [17]:
def find_max_job_id(start_id):
    current_id = start_id
    while True:
        url = f"{BASE_URL}{current_id + 1}"
        if is_valid_job_posting(url):
            current_id += 1
            logging.debug(f"Valid job posting found at ID {current_id}")
            print(current_id)
        else:
            logging.info(f"Max job ID found: {current_id}")
            break
    return current_id

In [15]:
max_id

1155